# House Price

First, i import the modules that i'll need to make the predictions

In [3]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor

Load the datasets, train e test

In [ ]:
# Read the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Get the value of the house, wich is the Y value, the value the we expected to predict.

In [ ]:
train_y = train.SalePrice

The function **set_encode** change the columns that have categorical values to numerical value, because the predict model only expect numerical values

In [ ]:
def set_encode(data):
    le = LabelEncoder()
    for column in data.select_dtypes(include=['object']):
        column_value = data[column].fillna('')
        le.fit(column_value)
        data[column] = le.transform(column_value)
        data[column][le.inverse_transform(data[column]) == ''] = np.nan

    return data

So pass the train e test dataset to the function

In [ ]:
train_X = set_encode(train_X)
test_X = set_encode(test_X)

With the code below i'm handling with the missing values on the datasets

In [ ]:
# Preprocessing
imputed_X_train_plus = train_X.copy()
imputed_X_test_plus = test_X.copy()

cols_with_missing = (col for col in train_X.columns
                     if train_X[col].isnull().any())


for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull()
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()

# Imputation
my_imputer = Imputer()
imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus)
imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)

Now i only need to create the model and train

In [ ]:
# Fit the model
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)

my_model.fit(imputed_X_train_plus, train_y, early_stopping_rounds=5,
             eval_set=[(imputed_X_train_plus, train_y)], verbose=False)

With the model already trained, i make the predictions

In [ ]:
# Use the model to make predictions
predicted_prices = my_model.predict(imputed_X_test_plus)

my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': predicted_prices})

# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)